In [1]:
import numpy as np

# Sigmoid函数
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# 卷积层
class ConvolutionalLayer:
    def __init__(self, input_shape, num_filters, kernel_size):
        self.input_shape = input_shape
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.filters = np.random.randn(num_filters, kernel_size, kernel_size) / (kernel_size ** 2)
        self.bias = np.zeros(num_filters,)
    
    def forward(self, input_data):
        self.input_data = input_data
        
        n, h_prev, w_prev, c_prev = input_data.shape
        h_out = h_prev - self.kernel_size + 1
        w_out = w_prev - self.kernel_size + 1
        
        output_data = np.zeros((n, h_out, w_out, self.num_filters))
        
        for i in range(h_out):
            for j in range(w_out):
                data_slice = input_data[:, i:i+self.kernel_size, j:j+self.kernel_size, :]
                for k in range(self.num_filters):
                    output_data[:, i, j, k] = np.sum(data_slice * self.filters[k], axis=(1, 2, 3))
            
        output_data += self.bias.reshape(1, 1, 1, self.num_filters)
        self.output_data = output_data
        
        return output_data
    
    def backward(self, output_error, learning_rate):
        input_error = np.zeros(self.input_shape)
        n, h_out, w_out, c_out = output_error.shape
        
        for i in range(h_out):
            for j in range(w_out):
                for k in range(self.num_filters):
                    input_error[:, i:i+self.kernel_size, j:j+self.kernel_size, :] += \
                        output_error[:, i:i+1, j:j+1, k:k+1] * self.filters[k]
                    self.filters[k] -= learning_rate * np.sum(
                        output_error[:, i:i+1, j:j+1, k:k+1] * self.input_data[:, i:i+self.kernel_size, j:j+self.kernel_size, :],
                        axis=0)
                    self.bias[k] -= learning_rate * np.sum(output_error[:, i:i+1, j:j+1, k:k+1])
        
        return input_error

# 池化层
class PoolingLayer:
    def __init__(self, pool_size):
        self.pool_size = pool_size
    
    def forward(self, input_data):
        self.input_data = input_data
        
        n, h_prev, w_prev, c_prev = input_data.shape
        h_out = h_prev // self.pool_size
        w_out = w_prev // self.pool_size
        
        output_data = np.zeros((n, h_out, w_out, c_prev))
        
        for i in range(h_out):
            for j in range(w_out):
                data_slice = input_data[:, i*self.pool_size:(i+1)*self.pool_size, j*self.pool_size:(j+1)*self.pool_size, :]
                output_data[:, i, j, :] = np.max(data_slice, axis=(1, 2))
        
        self.output_data = output_data
        
        return output_data
    
    def backward(self, output_error):
        input_error = np.zeros(self.input_data.shape)
        n, h_out, w_out, c_out = output_error.shape
        
        for i in range(h_out):
            for j in range(w_out):
                data_slice = self.input_data[:, i*self.pool_size:(i+1)*self.pool_size, j*self.pool_size:(j+1)*self.pool_size, :]
                max_indices = np.argmax(data_slice, axis=(1, 2))
                dmax = np.zeros((n, self.pool_size * self.pool_size, c_out))
                dmax[np.arange(n), max_indices.flatten(), np.repeat(np.arange(c_out), self.pool_size * self.pool_size)] = \
                    output_error[:, i, j, :].flatten()
                input_error[:, i*self.pool_size:(i+1)*self.pool_size, j*self.pool_size:(j+1)*self.pool_size, :] += \
                    dmax.reshape((n, self.pool_size, self.pool_size, c_out))
        
        return input_error

# 全连接层
class DenseLayer:
    def __init__(self, input_shape, units):
        self.input_shape = input_shape
        self.units = units
        self.weights = np.random.randn(input_shape, units) * np.sqrt(2 / input_shape)